In [22]:
#라이브러리 import

import requests
from bs4 import BeautifulSoup
import pandas as pd

requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

#요청url 잘게 자르기
url = "https://krdict.korean.go.kr/api/search?"
serviceKey = "certkey_no=3349&key=EAE8B2C9214D808997D177C50333BBFA"
typeOfSearch="&type_search=search"
part = "&part=word"
word = "인재"
sort = "&sort=popular"

def parse():
    try:
        TGT = item.find("target_code").get_text()
        WORD = item.find("word").get_text()
        DEF = item.find("definition").get_text()
        return {
              "코드":TGT,
              "단어":WORD,
              "뜻":DEF,
              }
       
    except AttributeError as e:
        return {
            "코드":None,
            "단어":None,
            "뜻":None,
        }
def target_parse():
    try:
        TGT = item.find("target_code").get_text()
        return {
              TGT,
              }
       
    except AttributeError as e:
        return {
            None,
        }
#parsing 하기
result = requests.get(url+serviceKey+typeOfSearch+part+"&q="+word+sort, verify=False)
soup = BeautifulSoup(result.text,'lxml-xml')
items = soup.find_all("item")
row = []
for item in items:
    WORD = item.find("word").get_text()
    if WORD == word:
      row.append(parse())

target_row = []
for item in items:
    WORD = item.find("word").get_text()
    if WORD == word:
      target_row.append(str(target_parse())[2:7])

#용례 가져오기
url2 = "https://krdict.korean.go.kr/api/view?"
serviceKey2 = "certkey_no=3349&key=EAE8B2C9214D808997D177C50333BBFA"
typeOfSearch2 = "&type_search=view"
method = "&method=TARGET_CODE"
part2 = "&part=word"
sort2 = "&sort=popular"

def parse2():
    try:
        WORD = item.find("target_code").get_text()
        EX = item.find("example").get_text()
        return {
              "단어":WORD,
              "용례":EX,
              }
       
    except AttributeError as e:
        return {
            "단어":None,
            "용례":None,
        }
row2 = []
for code in target_row:
  word2 = code
  result2 = requests.get(url2+serviceKey2+typeOfSearch2+method+part2+"&q="+word2+sort2, verify=False)
  soup = BeautifulSoup(result2.text,'lxml-xml')
  items2 = soup.find_all("item")

for item in items2:
    row2.append(parse2())
#용례 보기 만들기
row3 = []
for item in row2:
  row3.append(str(item)[22:])
row4 = []
for item in row3:
  val = item.find(word)
  string = item[:val] + "____" + item[val+len(word):]
  row4.append(string)

In [23]:
#단어 코드(동음이의어 시 2개 이상)
print(target_row)

['25092', '72531']


In [24]:
#단어 코드, 단어, 뜻(동음이의어 시 2개 이상)
print(row)

[{'코드': '25092', '단어': '인재', '뜻': '학식과 능력을 갖추어 사회적으로 크게 쓸모가 있는 사람.'}, {'코드': '72531', '단어': '인재', '뜻': '사람에 의해 일어난 불행한 사고나 괴롭고 어려운 일.'}]


In [25]:
# 단어코드, 용례(동음이의어 시 2개 이상)
print(row2)

[{'단어': '72531', '용례': '직원들이 난로를 끄지 않고 퇴근해 회사에 불이 나는 인재가 발생했다.'}]


In [26]:
print(row3)

["'직원들이 난로를 끄지 않고 퇴근해 회사에 불이 나는 인재가 발생했다.'}"]


In [27]:
#용례 빈칸 문제 보기
print(row4)

["'직원들이 난로를 끄지 않고 퇴근해 회사에 불이 나는 ____가 발생했다.'}"]
